In [1]:
from simpleaichat import AIChat
import pandas as pd
import textwrap

In [2]:
df = pd.read_csv('MACONLINE_rubros_prueba_T.csv')
df.head()

,comunidad,rubro,media_global,media_comunidad,t_score,p_value
0,C0,Agrupaciones sociales y profesionales,0.212229,0.336157,6.844169,8.111435e-12
1,C0,Alcohol y Entretenimiento,1.761468,3.025880,18.837505,6.655561e-78
2,C0,Alimentacion,31.933285,38.555297,20.317356,4.400288e-90
3,C0,Animales y Mascotas,0.782158,0.998045,8.907787,6.045562e-19
4,C0,Artes y Hobbies,0.998173,1.340160,13.985100,4.674301e-44


In [3]:
lifestyles = list(df['rubro'].unique())
lifestyles

['Agrupaciones sociales y profesionales',
 'Alcohol y Entretenimiento',
 'Alimentacion',
 'Animales y Mascotas',
 'Artes y Hobbies',
 'Articulos Electronicos',
 'Articulos de oficina',
 'Articulos electronicos',
 'Articulos y servicios del hogar',
 'Automotriz',
 'Belleza y Cuidado Personal',
 'Compras y Retail',
 'Construccion y grandes obras',
 'Cultura y Entretenimiento',
 'Deportes y Actividades Fisicas',
 'Educación',
 'Gastronomía',
 'Inmobiliarios',
 'Instituciones financieras y legales',
 'Jardinería y Naturaleza',
 'Juegos de azar',
 'Maternidad',
 'Salud y Bienestar',
 'Servicios Basicos',
 'Tercera edad',
 'Tiendas de conveniencia',
 'Transporte',
 'Turismo y Viajes',
 'Vestuario y Calzado']

In [4]:
communities = list(df['comunidad'].unique())
communities

['C0', 'C99', 'C2', 'C1']

In [5]:
result = []

for community in communities:
    df_community = df[df['comunidad'] == community].sort_values(by=['t_score'], ascending=False)
    top_lifestyles = list(df_community['rubro'])[:3]
    bottom_lifestyles = list(df_community['rubro'])[-3:]

    result.append([community, top_lifestyles, bottom_lifestyles])
    

In [6]:
df_result = pd.DataFrame(result, columns=['community', 'top_lifestyles', 'bottom_lifestyles'])
df_result

,community,top_lifestyles,bottom_lifestyles
0,C0,"[Gastronomía, Transporte, Automotriz]","[Jardinería y Naturaleza, Tercera edad, Matern..."
1,C99,"[Tercera edad, Educación, Articulos electronicos]","[Transporte, Gastronomía, Alimentacion]"
2,C2,"[Salud y Bienestar, Vestuario y Calzado, Compr...","[Tiendas de conveniencia, Gastronomía, Alcohol..."
3,C1,"[Alimentacion, Alcohol y Entretenimiento, Auto...","[Juegos de azar, Vestuario y Calzado, Turismo ..."


In [7]:
from pydantic import BaseModel, Field

def get_buyer_persona(response):
    ai_persona = AIChat(
        console=False,
        save_messages=False,  # with schema I/O, messages are never saved
        model="gpt-4",
        params={"temperature": 0.0},
    )

    class get_persona_metadata(BaseModel):
        """Event information"""

        name: str = Field(description="Full name of the person")
        age: str = Field(description="Age of the buyer persona")
        ocuppation: str = Field(description="Ocuppation of the buyer persona")
        description: str = Field(description="Summarized description of the buyer persona")
        pains: str = Field(description="Pains points of the buyer persona")
        buying_motivations: str = Field(description="Buying motivations of the buyer persona")
        buying_list: str = Field(description="Example shopping list of the buyer persona")

    # returns a dict, with keys ordered as in the schema
    persona = ai_persona(response, output_schema=get_persona_metadata)

    return persona

In [26]:
def get_data(top_lifestyles, bottom_lifestyles, gender):
    SYSTEM_PROMPT = f"""
                You are a market researcher. You are studying the lifestyles of people in different communities.
                You have data about what is the most popular buying habits, including the purchase categories of credit cards transactions. These categories are: {', '.join(lifestyles)}.
                """
    
    USER_PROMPT = f"""
                Give information on the demographics of people most likely to buy in these categories (ordered by importance): {', '.join(top_lifestyles)}.
                Only give one demographic group that englobes all the categories at once. Don't use gender to describe the group.
                Please respond in spanish.
            """
    
    ai = AIChat(system=SYSTEM_PROMPT, model="gpt-4")#gpt-3.5-turbo
    description = ai(USER_PROMPT)

    title = ai("Create a simple but catchy name for this demographic group that englobes all the categories at once. Respond in spanish. Only respond with the name of the group.")

    response = ai(f"Create a detailed buyer persona for this demographic group. Use {gender} gender. Include a full name (first name and last name) Include pains, buying motivations and a example shopping list. Output in markdown format. Respond in spanish.")

    result_dict = get_buyer_persona(response)

    #add 'description' and 'title' to 'result_dict'
    result_dict['title'] = title
    result_dict['community_description'] = description

    return result_dict
    


## Comunidad 0

In [27]:
top_lifestyles = df_result.iloc[0]['top_lifestyles']
bottom_lifestyles = df_result.iloc[0]['bottom_lifestyles']
gender = 'female'

In [28]:
result_dict  = get_data(top_lifestyles, bottom_lifestyles, gender)

result_dict

In [ ]:
# Write 'result_dict' to a txt file
with open('comunidad-C0.txt', 'w') as f:
    f.write(str(result_dict))

## Comunidad 1

In [ ]:
top_lifestyles = df_result.iloc[2]['top_lifestyles']
bottom_lifestyles = df_result.iloc[2]['bottom_lifestyles']
gender = 'male'

In [ ]:
result_dict = get_data(top_lifestyles, bottom_lifestyles, gender)

result_dict

{'name': 'Alejandro Martínez',
 'age': '30',
 'ocuppation': 'Ingeniero de Software',
 'description': 'Alejandro es un ingeniero de software de 30 años que vive en Barcelona, España. Es soltero y tiene un ingreso anual de €60,000. Lucha por equilibrar su ajetreada vida profesional con su deseo de mantener una vida saludable. A menudo tiene poco tiempo para ir de compras, por lo que valora las experiencias de compra eficientes y fiables. A veces se siente abrumado por la cantidad de opciones disponibles en el mercado y busca marcas y productos de confianza.',
 'pains': '1. Alejandro lucha por equilibrar su ajetreada vida profesional con su deseo de mantener una vida saludable.\n2. A menudo tiene poco tiempo para ir de compras, por lo que valora las experiencias de compra eficientes y fiables.\n3. A veces se siente abrumado por la cantidad de opciones disponibles en el mercado y busca marcas y productos de confianza.',
 'buying_motivations': '1. Alejandro valora su salud y bienestar y est

In [ ]:
with open('comunidad-C1.txt', 'w') as f:
    f.write(str(result_dict))

## Comunidad 2

In [ ]:
top_lifestyles = df_result.iloc[3]['top_lifestyles']
bottom_lifestyles = df_result.iloc[3]['bottom_lifestyles']
gender = 'female'

In [ ]:
result_dict = get_data(top_lifestyles, bottom_lifestyles, gender)

result_dict

{'name': 'Laura González',
 'age': '28',
 'ocuppation': 'Ingeniera',
 'description': 'Laura González es una joven profesional de 28 años que vive en el centro de una ciudad grande. Trabaja como ingeniera en una importante empresa de tecnología y tiene un salario decente.',
 'pains': 'Laura lleva un estilo de vida ocupado y a menudo lucha por encontrar tiempo para comprar y cocinar. A veces, gasta más de lo que le gustaría en entretenimiento y socialización. Aunque tiene un coche, los gastos de mantenimiento y combustible pueden ser altos y a menudo se pregunta si hay una alternativa más económica y sostenible.',
 'buying_motivations': 'Laura valora la comodidad y está dispuesta a pagar más por productos y servicios que le ahorren tiempo. Le gusta socializar y disfrutar de experiencias de entretenimiento de calidad, por lo que a menudo gasta en restaurantes, bares y eventos. Necesita su coche para desplazarse al trabajo y a las actividades sociales, por lo que está dispuesta a invertir 

In [ ]:
with open('comunidad-C2.txt', 'w') as f:
    f.write(str(result_dict))

## Comunidad 4

In [199]:
top_lifestyles = df_result.iloc[4]['top_lifestyles']
bottom_lifestyles = df_result.iloc[4]['bottom_lifestyles']
gender = 'male'

In [200]:
result_dict = get_data(top_lifestyles, bottom_lifestyles, gender)

result_dict

{'name': 'Juan Martínez',
 'age': '30 años',
 'ocuppation': 'Profesional en marketing',
 'description': 'Juan Martínez es un hombre joven y apasionado por la gastronomía, la alimentación saludable y el cuidado personal. Busca constantemente opciones que le permitan mantener un estilo de vida equilibrado y saludable. Está dispuesto a invertir en productos y servicios que le ayuden a lograr sus objetivos de bienestar. Juan se informa a través de diferentes canales y busca constantemente nuevas experiencias gastronómicas y opciones que se ajusten a sus necesidades de alimentación saludable.',
 'pains': '- Dificultad para encontrar opciones de restaurantes y productos que se ajusten a sus necesidades de alimentación saludable.\n- Estrés y falta de tiempo para planificar y preparar comidas saludables en casa.\n- Preocupación por mantener un estilo de vida equilibrado y encontrar productos de cuidado personal que sean naturales y de calidad.',
 'buying_motivations': '- Busca productos y serv

In [201]:
with open('comunidad-4.txt', 'w') as f:
    f.write(str(result_dict))